In [ ]:
import sys
import numpy as np
#from matplotlib.mlab import PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

data = my_data = np.genfromtxt('full_pcm.results', delimiter=',')
#remove header and non-feature column
X = data[1:,4:]
Y = data[1:,3]

X = normalize(X, axis=0, norm='l2')


In [ ]:

gamma_list=[10, 20, 40]
C_list=[10]
# cross validation 
for g in gamma_list:
    for c in C_list:
        true_positive_count= 0
        false_positive_count= 0
        false_negative_count= 0
        for i in range(28):
            train_X = np.concatenate((X[:i*8,:],X[i*8+8:,:]))
            test_X = X[i*8:i*8+8,:]
            train_Y = np.concatenate((Y[:i*8],Y[i*8+8:]))
            test_Y = Y[i*8:i*8+8]
            pca = PCA(n_components=5)
            pca.fit(train_X)
            #print(pca.explained_variance_ratio_)

            transformed_train_X = pca.transform(train_X)
            transformed_test_X = pca.transform(test_X)
            #pca.components_

            #learning

            #learner = svm.SVC(kernel='linear', gamma=g, C=c, class_weight={0:0.5, 1:0.8})
            #learner = RandomForestClassifier(n_estimators=c, n_jobs=7, max_depth=g,class_weight={0:0.5, 1:5})
            #learner = KNeighborsClassifier(n_neighbors=20)
            #learner = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10,class_weight={0:0.5, 1:5}), algorithm="SAMME", n_estimators=1000)
            #learner = ExtraTreesClassifier(n_estimators=10, n_jobs=7, criterion='entropy', class_weight={0:0.5, 1:10})
            learner = LogisticRegression(solver='lbfgs', class_weight={0:1, 1:1.8})
            learner.fit(transformed_train_X,train_Y)

            predicted=learner.predict(transformed_test_X)
            diff = predicted - test_Y

            for index in range(len(predicted)):
                if predicted[index] ==0 and test_Y[index] ==0:
                    true_positive_count +=1
                if predicted[index] ==0 and test_Y[index] ==1:
                    false_positive_count +=1
                if predicted[index] ==1 and test_Y[index] ==0:
                    false_negative_count += 1
            #print(np.absolute(diff))
        print("C = " + str(c) + ", gamma = " + str(g))
        print("recall = " + str(true_positive_count/(true_positive_count+false_negative_count)))
        print("precision = " + str(true_positive_count / (true_positive_count + false_positive_count)))
        print("\n")